In [39]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatx=float32 python cifar10_cnn.py
It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 25

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', use_bias=False, input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu', use_bias=False))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', use_bias=False))
model.add(Conv2D(64, (3, 3), activation='relu', use_bias=False))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# conv10 output shape = (?, 13, 13, 6)
model.add(Conv2D(num_classes, (1, 1), activation='relu', use_bias=False))
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save('cifar10_base')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 161s - loss: 1.8160 - acc: 0.3185 - val_loss: 1.5720 - val_acc: 0.4330
Epoch 2/25
50000/50000 [==============================] - 159s - loss: 1.4818 - acc: 0.4639 - val_loss: 1.4555 - val_acc: 0.4808
Epoch 3/25
50000/50000 [==============================] - 169s - loss: 1.3213 - acc: 0.5299 - val_loss: 1.2428 - val_acc: 0.5498
Epoch 4/25
50000/50000 [==============================] - 207s - loss: 1.2092 - acc: 0.5718 - val_loss: 1.1412 - val_acc: 0.5983
Epoch 5/25
50000/50000 [==============================] - 265s - loss: 1.1305 - acc: 0.6049 - val_loss: 1.1481 - val_acc: 0.5935
Epoch 6/25
50000/50000 [==============================] - 267s - loss: 1.0692 - acc: 0.6260 - val_loss: 1.1270 - val_acc: 0.6194
Epoch 7/25
50000/50000 [==============================] - 322s - loss: 1.0178 - acc: 0.6463 - val_loss: 0.9846 - val_acc: 0.6559
Epoch 8/25
50000/50000 [=======================

In [40]:
model.save('cifar10_globalavg_25ep')
# model.layers

In [34]:
np.array(model.layers[0].get_weights()).shape

(1, 3, 3, 3, 32)

In [38]:
np.array(model.layers[4].get_weights()).shape

(1, 3, 3, 32, 64)

In [33]:
np.array(model.layers[1].get_weights())

array([[[[[  2.26638600e-01,  -3.66327196e-01,  -2.70816367e-02, ...,
            -3.30585986e-01,  -4.42860007e-01,   1.03598922e-01],
          [  2.51502812e-01,   7.30787888e-02,   2.46326759e-01, ...,
             2.29454368e-01,  -1.74416155e-01,   6.97132722e-02],
          [  5.18631637e-02,   7.32588321e-02,   1.89953238e-01, ...,
            -2.94342786e-01,  -1.11988463e-01,  -3.85406911e-01],
          ..., 
          [  6.06607944e-02,  -7.42304385e-01,  -6.87854216e-02, ...,
            -1.34996414e-01,   9.88706797e-02,  -2.40444481e-01],
          [  3.60393859e-02,  -1.76986724e-01,  -1.24336556e-02, ...,
            -6.89748256e-03,   2.42830172e-01,  -2.89227486e-01],
          [  4.99344654e-02,   1.93324104e-01,   4.41755839e-02, ...,
             1.79024056e-01,  -3.02707702e-02,  -2.18332157e-01]],

         [[  3.11181664e-01,  -3.30501348e-02,  -1.55259326e-01, ...,
            -3.11373055e-01,  -3.20470840e-01,   2.30225235e-01],
          [  1.89826623e-01,  